In [ ]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
# 데이터 로드
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')
df_sample_submission = pd.read_csv('./sample_submission.csv')

In [ ]:
# ID 제거
df_train.drop(columns=['ID'], inplace=True)
df_test_ids = df_test['ID']
df_test.drop(columns=['ID'], inplace=True)

In [ ]:
# 타겟 변수 분리
X = df_train.drop(columns=['임신 성공 여부'])
y = df_train['임신 성공 여부']

In [ ]:
# 🔹 결측치 확인 및 처리
print("📊 결측치 개수:")
print(X.isnull().sum())

📊 결측치 개수:
시술 시기 코드                      0
시술 당시 나이                      0
임신 시도 또는 마지막 임신 경과 연수    246981
시술 유형                         0
특정 시술 유형                      2
                          ...  
난자 채취 경과일                 57488
난자 해동 경과일                254915
난자 혼합 경과일                 53735
배아 이식 경과일                 43566
배아 해동 경과일                215982
Length: 67, dtype: int64


In [ ]:
# 숫자형 변수 결측치 평균 대체
numeric_features = X.select_dtypes(include=[np.number]).columns
imputer = SimpleImputer(strategy="mean")
X[numeric_features] = imputer.fit_transform(X[numeric_features])
df_test[numeric_features] = imputer.transform(df_test[numeric_features])

In [ ]:
# 범주형 변수 인코딩 (Ordinal Encoding)
categorical_features = X.select_dtypes(include=['object']).columns
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X[categorical_features] = encoder.fit_transform(X[categorical_features])
df_test[categorical_features] = encoder.transform(df_test[categorical_features])

In [ ]:
# 🔹 Feature Scaling (StandardScaler 적용) → 먼저 적용
scaler = StandardScaler()
X[numeric_features] = scaler.fit_transform(X[numeric_features])
df_test[numeric_features] = scaler.transform(df_test[numeric_features])


In [ ]:
# 🔹 Feature Selection (LightGBM 기반, 임계값 조정)
lgbm = LGBMClassifier(n_estimators=200, random_state=42, n_jobs=-1)
lgbm.fit(X, y)
feature_importance = pd.Series(lgbm.feature_importances_, index=X.columns).sort_values(ascending=False)
selected_features = feature_importance[feature_importance > 0.0005].index.tolist()
X = X[selected_features]
df_test = df_test[selected_features]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.093993 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 771
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 60
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054568
[LightGBM] [Info] Start training from score -1.054568


In [ ]:
# 🔹 Feature Selection 이후 NaN 체크 및 재처리
imputer = SimpleImputer(strategy="mean")
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
df_test = pd.DataFrame(imputer.transform(df_test), columns=df_test.columns)

In [ ]:
# 훈련 / 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 🔹 Optuna 최적화 함수 정의
def objective(trial):
    # 개별 모델의 하이퍼파라미터 튜닝
    lgbm_params = {
        'n_estimators': trial.suggest_int('lgbm_n_estimators', 300, 500),
        'learning_rate': trial.suggest_loguniform('lgbm_learning_rate', 0.003, 0.015),
        'num_leaves': trial.suggest_int('lgbm_num_leaves', 30, 100),
        'max_depth': trial.suggest_int('lgbm_max_depth', 4, 12),
        'subsample': trial.suggest_float('lgbm_subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('lgbm_colsample_bytree', 0.5, 1.0),
        'random_state': 42
    }

    xgb_params = {
        'n_estimators': trial.suggest_int('xgb_n_estimators', 300, 500),
        'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 0.003, 0.015),
        'max_depth': trial.suggest_int('xgb_max_depth', 4, 12),
        'subsample': trial.suggest_float('xgb_subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('xgb_colsample_bytree', 0.5, 1.0),
        'random_state': 42
    }

    catboost_params = {
        'iterations': trial.suggest_int('catboost_iterations', 300, 500),
        'learning_rate': trial.suggest_loguniform('catboost_learning_rate', 0.003, 0.015),
        'depth': trial.suggest_int('catboost_depth', 4, 12),
        'l2_leaf_reg': trial.suggest_float('catboost_l2_leaf_reg', 2, 10),
        'random_state': 42,
        'verbose': 0
    }

    # 🔹 개별 모델 정의 (Optuna 튜닝 적용)
    lgbm = LGBMClassifier(**lgbm_params)
    xgb = XGBClassifier(**xgb_params)
    catboost = CatBoostClassifier(**catboost_params)

    # 🔹 Meta Model 선택 (Logistic Regression vs Random Forest)
    meta_model_choice = trial.suggest_categorical('meta_model', ['logistic', 'random_forest'])

    if meta_model_choice == 'logistic':
        meta_model = LogisticRegression()
    else:
        meta_model = RandomForestClassifier(n_estimators=100, random_state=42)

    # 🔹 Stacking 모델 설정
    stacked_model = StackingClassifier(
        estimators=[
            ('lgbm', lgbm),
            ('xgb', xgb),
            ('catboost', catboost)
        ],
        final_estimator=meta_model,
        stack_method='predict_proba',
        passthrough=True,
        n_jobs=-1
    )

    # 🔹 Stacking 학습
    stacked_model.fit(X_train, y_train)

    # 🔹 검증 데이터 평가
    y_val_pred = stacked_model.predict_proba(X_val)[:, 1]
    return roc_auc_score(y_val, y_val_pred)

In [ ]:
# 🔹 Optuna 실행
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=30)

[I 2025-02-04 11:18:06,610] A new study created in memory with name: no-name-8daaa62a-b808-435e-930f-815253868960
<ipython-input-14-9713eceb3535>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('lgbm_learning_rate', 0.003, 0.015),
<ipython-input-14-9713eceb3535>:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 0.003, 0.015),
<ipython-input-14-9713eceb3535>:25: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  '

In [ ]:
# 🔹 최적의 하이퍼파라미터 출력
best_params = study.best_params
print(f"🔹 Best Params: {best_params}")

🔹 Best Params: {'lgbm_n_estimators': 436, 'lgbm_learning_rate': 0.014768177339829039, 'lgbm_num_leaves': 87, 'lgbm_max_depth': 12, 'lgbm_subsample': 0.6628572716745862, 'lgbm_colsample_bytree': 0.7129596637133632, 'xgb_n_estimators': 347, 'xgb_learning_rate': 0.006945354657061471, 'xgb_max_depth': 10, 'xgb_subsample': 0.5033839027206597, 'xgb_colsample_bytree': 0.5110990994971739, 'catboost_iterations': 300, 'catboost_learning_rate': 0.011409195641314061, 'catboost_depth': 12, 'catboost_l2_leaf_reg': 9.955027922167531, 'meta_model': 'logistic'}


In [ ]:
# 🔹 최적의 Stacking 모델 생성
final_lgbm = LGBMClassifier(
    n_estimators=best_params['lgbm_n_estimators'],
    learning_rate=best_params['lgbm_learning_rate'],
    num_leaves=best_params['lgbm_num_leaves'],
    max_depth=best_params['lgbm_max_depth'],
    subsample=best_params['lgbm_subsample'],
    colsample_bytree=best_params['lgbm_colsample_bytree'],
    random_state=42
)

final_xgb = XGBClassifier(
    n_estimators=best_params['xgb_n_estimators'],
    learning_rate=best_params['xgb_learning_rate'],
    max_depth=best_params['xgb_max_depth'],
    subsample=best_params['xgb_subsample'],
    colsample_bytree=best_params['xgb_colsample_bytree'],
    random_state=42
)

final_catboost = CatBoostClassifier(
    iterations=best_params['catboost_iterations'],
    learning_rate=best_params['catboost_learning_rate'],
    depth=best_params['catboost_depth'],
    l2_leaf_reg=best_params['catboost_l2_leaf_reg'],
    random_state=42,
    verbose=0
)

In [ ]:
# 🔹 최적 Meta Model 설정
if best_params['meta_model'] == 'logistic':
    final_meta_model = LogisticRegression()
else:
    final_meta_model = RandomForestClassifier(n_estimators=100, random_state=42)

# 🔹 최적 Stacking 모델 학습
final_stacked_model = StackingClassifier(
    estimators=[
        ('lgbm', final_lgbm),
        ('xgb', final_xgb),
        ('catboost', final_catboost)
    ],
    final_estimator=final_meta_model,
    stack_method='predict_proba',
    passthrough=True,
    n_jobs=-1
)

final_stacked_model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(estimators=[('lgbm',
                                LGBMClassifier(colsample_bytree=0.7129596637133632,
                                               learning_rate=0.014768177339829039,
                                               max_depth=12, n_estimators=436,
                                               num_leaves=87, random_state=42,
                                               subsample=0.6628572716745862)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.51109...
                                              max_delta_step=None, max_depth=10,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=347, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=42, ...)),
                               ('catboost',
                                <catboost.core.CatBoostClassifier object at 0x7f2fd91da010>)],
                   final_estimator=LogisticRegression(), n_jobs=-1,
                   passthrough=True, stack_method='predict_proba')

In [ ]:
# 🔹 검증 데이터 평가
y_val_pred = final_stacked_model.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_val_pred)
print(f'✅ Validation ROC-AUC (Stacking + Optuna): {roc_auc:.4f}')

✅ Validation ROC-AUC (Stacking + Optuna): 0.7353


In [ ]:
# 🔹 테스트 데이터 예측
test_preds = final_stacked_model.predict_proba(df_test)[:, 1]

In [ ]:
# 7. 제출 파일 생성
submission = pd.DataFrame({'ID': df_test_ids, 'probability': test_preds})
submission.to_csv('submission_7.csv', index=False)
print("✅ Submission file saved: submission_7.csv")

✅ Submission file saved: submission_7.csv
